## 이분탐색 중 징검다리 문제

# 문제링크: https://programmers.co.kr/learn/courses/30/lessons/43236

문제 읽기: 0부터 distance 사이에 rock들이 있음  
이 중 n개 제거했을 때 각 사이거리 중 최소값을 뽑음  
제거하는 경우의 수마다 최솟값이 다를텐데, 이 중 최대값을 반환  
  
입력값: distance, rocks, n  
출력값: 각 지점 사이 거리의 최소값 중 최대값?  

제한사항:    
1<= distance <= 1,000,000,000    
1<= k <= 50,000 ( k는 총 바위 개수로 rocks의 원소수라 가정 )    
1<= n <= k  

# 해결 코드:

In [1]:
#21.09.10 100%

def valid_val_checker( candiVal, distance, rocks, n):
    
    rocks = [0] + rocks
    
    idx = len(rocks) - 1
    end_point = distance
    while True:
        
        print("     idx: ", idx, "  rocks : ", rocks, " n: ", n, " end_point: ", end_point)
        
        if end_point - rocks[idx] < candiVal: #돌 제거 작업
            n -= 1
            rocks.pop()
            idx -= 1
        else: #제거가 필요 없다면 다음 돌로 이동!
            end_point = rocks[idx]
            idx -= 1           
        
        
        if n < 0: return False
        
        if idx < 0: break
    
    return True


def solution(distance, rocks, n): #acc100
    answer = 0
    rocks, rocks_len = sorted(rocks), len(rocks)
    
    print(rocks)
    
    remove_cnt = n
    
    #binary search로 0~distance 사이의 원하는 값 val 찾기
    start, end = 0, distance
    
    candidates = set()
    past_mid = 1000000001 #될 수 없는 수로 초기화
    while True:
    
        mid = int( (start + end) / 2 )
        print(" s / e / m / past m : ", start, end, mid, past_mid)
        if mid == past_mid: break 
        
        if valid_val_checker(mid, distance, rocks, n ) :
            candidates.add(mid)
            start = mid
        else:
            end = mid
        
        if start == end: break
            
        past_mid = mid
        
        
    ###val이 최소값이 되도록 linear하게 rocks 중 val 보다 작은 값들 제거
    
    ###제거 카운트 초과하면 잘못된 값인 것임. val 값 줄여서 다시 시작
    print(" candidates : ",candidates)
    answer = max(candidates)
    
    
    return answer


# 해설:

너무나 시행착오가 많았다.  
정답은 아래 한 줄 읽으면 구할 수 있음  
0과 distance 사이에서 answer 찾고 rocks 리스트에서 answer 보다 크게하는 돌 삭제. 
바이너리서치로 answer 찾는데 판정함수로 answer의 candidate(n)를 이용하는 것이다. 
  
즉, 바이너리 서치에서 validation 함수를 사용하여 valid하면(n개 이하로 돌 없애도 answer이 가능) answer에 더 큰 값을 넣어보고  
valid하지 않으면 answer에 더 작은 값을 넣어보는 것이다.  
  
valid는 candidate 보다 작은 돌의 삭제를 parameter n 안에 해결 가능한 지를 체크하는 것!    
(-해결 가능하면 더 큰 answer가 있는지 체크,)  
(-아니면 더 작은 answer가 있는지 체크.)  
(바이너리 서치 플로우)  


시행착오를 정리해보자면?  
기본적으로 처음에 돌 삭제 보다는 range merge 방식으로 접근하였다.  
merge를 통해 최소값을 없애가는 방식..  
근데 최소값만을 삭제해가는 greedy한 방식으로는 최적해를 구하지 못 했다.  
그래서 combination으로 모든 경우를 찾아가는 방법도 생각했으나 시간초과.  
둘 다 안 되서 range merge가 아닌 rock pick 방식으로 접근하였다.  
분점을 이루는 돌을 찾는 방법. 뭐냐하면 2등분한다면 2분점을 만드는 가운데 돌 하나  
3등분 한다면 3분점을 만드는 돌 두개 찾기.  
비율을 조정해서 전체 길이의 분점을 찾아가는 방법으로 트라이하였다.   
예를 들어, ⅓ 점 ⅔ 점을 비율로 1:2가 되는 점을 찾는 것.  
근데 이 방법은 1:2 를 싸그리 무시할 수 있는 긴 range를 가진 경우 무력화되었다.  
비율이 망가져버려서 오답을 찾아버리고 마는 것이다.  
그래서 변주된 새로운 방법으로 2등분 찾고 3등분, 그 다음 4등분으로  
순차적으로 등분을 찾아가는 방법을 생각해보았다.  
예를들어,2등분 점을 딱 찾으면 구간은 [0, 2등분점], [2등분점, 총거리] 이렇게 나눠진다.  
그러면 두 구간들에서 각각 3등분점을 하나씩 구하면 된다.  
이러한 방식으로 쭉쭉 들어가서 원하는 등분점을 찾는 방법이다.  
이 방법의 경우 만약 비율로 다시 등분 점을 찾는다면 직전 방법과 다를 바가 없었다.  
그래서 판정함수로 min값을 리턴해주고 이 min값이 가장 큰 경우를 찾는 것을 만들었.  
하지만 판정함수가 구간이 작은 경우 잘 동작하지 않았다.  
예를 들어 [0, 1] 구간에서 rock은 [2,11]이고  
range는 [ (2, 11)    (11, 0)  ]이 된다.   
이 경우 1의 range가 0이 되어 무조건 0을 분점으로 선택하는 분제가 발생한다.  
해결을 위해 range의 범위를 끝 rock이 아닌 직전 분점으로 하는 방법도 구현하였으나..  
오히려 계산이 꼬여서 잘못된 값들이 도출되었다.  
그리고 가장 결정적으로 시간 복잡도가 nlogn을 초과해서, 시간 초과!  
그래서 결과적으로 힌트에 기인한 방법으로 문제를 해결하게 되었다.   
장장 한달간의 시행착오 스토리..  




# 다시 풀어 본 코드

In [1]:
def helper_func(min_dist, rocks, n): #acc100 211202
    #두바퀴 돌려야겠네
    
    rocks_new = rocks[:]
    rocks = rocks_new
    
    rock_num = len(rocks)
    i = 1
    sp = rocks[0]
    
    while True:
        #print("     sp is : %.d, i is : %.d,  n is : %.d, rock_num is : %.d , rocks :  "%(sp, i,n,rock_num), rocks)
        if n < 0:
            #print("1")
            return False
        
        if i == rock_num-1: break
        
        if rocks[i] - sp < min_dist:
            del rocks[i]
            rock_num -= 1
            n -= 1
        else:
            sp = rocks[i]
            i += 1
        
        
    
    if rocks[i] == sp or rocks[i] - rocks[i-1]: return True
    
    ep = i
    i = i - 1
    while True:
        #print("     sp is : %.d, i is : %.d,  n is : %.d, rock_num is : %.d , rocks :  "%(sp, i,n,rock_num), rocks)
        if n < 0: 
            #print("2")
            return False
        
        if i == 0: break
        
        if ep - rocks[i] < min_dist:
            del rocks[i]
            rock_num -= 1
            n -= 1
            i -= 1
        else:
            ep = rocks[i]
            i -= 1
        
        
    
    if ep == rocks[i] : return True
    if rocks[i+1] - rocks[i] >= min_dist :  return True
    else:
        #print("3")
        return False
        


def solution(distance, rocks, n):
    
    rocks = sorted(rocks)    
    rocks = [0] + rocks
    rocks.append(distance)
    #print(rocks)
    start = 1
    end = distance
    
    while True:
        mid = (start + end) // 2
        #print("start : %.d, end : %.d, mid : %.d, n is %.d, rocks : "%(start, end, mid, n), rocks)
        #print(helper_func(mid, rocks, n))
        
        if helper_func(mid, rocks, n) :
            
            #print("true?")
            #print(mid+1, rocks, n, "why something wrong ????? god")
            #print(helper_func(mid+1, rocks, n))
            if mid+1 < distance and helper_func(mid+1, rocks, n) is False: return mid
            else: start = mid
                
        else: 
            #print("false")
            end = mid

판정 함수 짜는 것이 너무 헬이었다..  

# 시행착오 코드: 

In [2]:
#21.08.17 55.6%
def solution(distance, rocks, n):
    answer = 0
    rocks.append(0)
    rocks.append(distance)
    rocks=sorted(rocks)
    dists = []
    length=len(rocks)
    for i in range(1, length):
        dist=rocks[i]-rocks[i-1]
        dists.append(dist)
    #print(dists)
    
    #좀 있다가 와일루프로 n만큼 돌릴 예정
    

    
                   
            
    for i in range(n):
        
        
        #find min value
        min_idxes = []
        min_val = min(dists)
        dist_length = len(dists)
        
        for j in range(dist_length):
            if dists[j] == min_val: 
                min_idxes.append(j)
                break #55%까지 올림....

#         if len(min_idxes) == 1: #걍 찾어서 merge
#             #좌우 체크는 해줘야함.
#             min_idx = min_idxes[0]
        #print("start!!!!!!!!!!start!!!!!!!!!")
        #print("min_val, min_idxes, dist, dist_length : ", min_val, min_idxes, dist, dist_length)
        if len(min_idxes) >= 1: #제일 merge값이 작은 경우를 merge시킴
            candidates = [] # 원소 구성: [merge_sum, min_idx, removable_idx]
            for min_idx in min_idxes:
                min_minus1_idx, min_plus1_idx, min_sum, removable_idx = -1, -1, 0, -1
                if min_idx - 1 > 0: min_minus1_idx = min_idx - 1        
                if min_idx + 1 < dist_length : min_plus1_idx = min_idx + 1

                if min_minus1_idx == -1 and min_plus1_idx == -1:
                    break
                elif min_minus1_idx != -1 and min_plus1_idx == -1:
                    
                    #뭐가문제일까
                    #print("min_idx, min_idxes, dists : ", min_idx, min_idxes, dists)
                    #print("   min_minus_1_idx, min_plus1_idx :",min_minus1_idx, min_plus1_idx)
                    
                    min_sum = dists[min_minus1_idx] + dists[min_idx]
                    removable_idx = min_minus1_idx
                elif min_minus1_idx == -1 and min_plus1_idx != -1:
                    min_sum = dists[min_idx] + dists[min_plus1_idx]
                    removable_idx = min_idx
                else: #min_minus1_idx != -1 and min_plus1_idx != -1:
                    if dists[min_minus1_idx] > dists[min_plus1_idx]: #더 작은 값으로 합쳐줌
                        min_sum = dists[min_idx] + dists[min_plus1_idx]
                        removable_idx = min_idx
                    else:
                        min_sum = dists[min_idx] + dists[min_minus1_idx]
                        removable_idx = min_minus1_idx
                candidates.append( [min_sum, min_idx, removable_idx] )
                #del dists[removable_idx]
                #dists[removable_idx] = min_sum #진짜 매우 복잡하네.....후우 
            
            candidate = min(candidates)
            
            #print(candidates, '  ',candidate)
            
            [min_sum, min_idx, removable_idx] = candidate
            
            del dists[removable_idx]
            dists[removable_idx] = min_sum

        else : # len(min_idxes) < 1 이면
            break

    answer = min(dists)
    
    
    
    
    return answer
